In [2]:
import openai
import pandas as pd
import numpy as np
import os
import json
import random
import nltk
from nltk.corpus import stopwords
from google_pygram import GooglePyGram as gpg
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
from string import punctuation

In [3]:
stoplist = set(stopwords.words('english'))

In [6]:
MODEL = "gpt-4-1106-preview"

In [7]:
samples = pd.read_json("sa_sample.jsonl", lines = True)

In [8]:
samples = samples.to_dict('records')

In [27]:
system_instruction = '''You are a linguist specializing in doing text annotation in the English language. You will be tasked with making minimal modification to a given piece of text based on some linguistics aspects to expose biases in machine learning models.

The given text are samples in the sentiment analysis task. 

Each sample has three fields: 
- Text: the input text

Make sure to keep the literal string value of Object the same when making modification.

A task may ask for one or multiple modifications. Each modification should be an object with 3 fields: 
type: the type of modification
modified_text: the modified text.
rationale: the reason why and how the modifications are made.

Please return a json object which consists of one or multiple modifications.
'''

In [10]:
def reformat_sample(sample):
    text = sample['text']
    sample['object'] = sample['label'][0][0]
    sample['span'] = sample['label'][0][1]
    
    return sample

In [ ]:
for i,sample in enumerate(samples):
    samples[i] = reformat_sample(sample)
    print(samples[i])

In [12]:
def format_prompt(system_instruction, instruction, input):
    # print(input)
    text = input['text']
    label = str(input['label'])
    object = input['object']
    span = input['span']
    # print(text)
    # print(label)
    message = instruction.format(text = text, object = object, span = span)
    messages = [
        {"role": "system", "content": system_instruction},
        {"role": "user", "content": message},
    ]
    return messages

In [13]:
# Example OpenAI Python library request
def request(samples, prompt_type):
    for sample in samples:
        messages = format_prompt(system_instruction, prompt_type, sample)
        response = openai.ChatCompletion.create(
            model=MODEL,
            response_format={ "type": "json_object" },
            messages= messages,
            temperature=0,
        )
        ans_model = response['choices'][0]['message']['content']
        # print(messages[1]['content'])
        print(sample)
        print(ans_model)
        print('===================')

Alternative requests and helper functions for the special cases which require another variable for randomization (geographical bias and nonce words)


Geographical bias

In [122]:
subsaharan_africa = ['Angola', 'Benin', 'Botswana', 'Burkina Faso', 'Burundi', 'Cameroon', 'Cape Verde',
          'Central African Republic', 'Chad', 'Comoros', 'Djibouti', 'Republic of the Congo', 'Democratic Republic of the Congo (Zaire)',
          "Côte d'Ivoire", 'Equatorial Guinea', 'Eritrea', 'Ethiopia', 'Gabon', 'Gambia', 'Ghana', 'Guinea', 
          'Guinea-Bissau', 'Kenya', 'Lesotho', 'Liberia', 'Madagascar', 'Malawi', 'Mali', 'Mauritius', 'Mozambique', 
          'Namibia', 'Niger', 'Nigeria', 'Rwanda', 'São Tomé and Príncipe', 'Senegal', 'Seychelles', 'Sierra Leone', 
          'Somalia', 'South Africa', 'South Sudan', 'Swaziland', 'Tanzania', 'Togo', 'Uganda']
          

#Middle East, North Africa, and Turkey		   
menat = ['Algeria', 'Bahrain', 'Iran', 'Iraq', 'Jordan', 'Egypt', 'Kuwait', 'Lebanon', 'Libya', 'Mauritania',
          'Morocco', 'Oman', 'Palestine', 'Qatar', 'Saudi Arabia', 'Sudan', 'Syria', 'Turkey',
          'Tunisia', 'United Arab Emirates', 'Yemen']


southeast_asia = ['Brunei', 'Cambodia', 'Timor Leste', 'Indonesia', 'Laos', 'Malaysia', 'Myanmar',
        'Philippines', 'Singapore', 'Thailand', 'Vietnam']

east_asia = ['China', 'Japan', 'Mongolia', 'North Korea', 'South Korea', 'Taiwan']

south_asia = ['Bangladesh', 'Bhutan', 'India', 'The Maldives', 'Nepal', 'Pakistan', 'Sri Lanka']

central_asia = ['Afghanistan', 'Armenia', 'Azerbaijan', 'Georgia', 'Kazakhstan', 'Kyrgyzstan', 
        'Tajikistan', 'Turkmenistan', 'Uzbekistan']


#Oceania, Melanesia, and Polynesia
oceania = ['Fiji', 'Federated States of Micronesia', 'Kiribati', 'Marshall Islands', 'Nauru', 
           'Palau', 'Papua New Guinea', 'Samoa', 'Solomon Islands', 'Tonga', 'Tuvalu', 'Vanuatu']

australia_nz = ['Australia', 'New Zealand']


#Carribean and Latin America
latin_america = ['Mexico', 'Puerto Rico', 'Dominican Republic', 'Cuba', 'Haiti', 'Belize', 'Grenada', 'Saint Lucia',
           'Costa Rica', 'El Salvador', 'Guatemala', 'Honduras', 'Nicaragua', 'Panama', 'Jamaica', 'Bahamas', 'Barbados',
           'Dominica', 'Brazil', 'Argentina', 'Bolivia', 'Chile', 'Colombia', 'Ecuador', 'Guyana', 'Paraguay', 'Peru',
           'Suriname', 'Trinidad and Tobago', 'Uruguay', 'Venezuela', 'Antigua and Barbuda', 'Saint Kitts and Nevis']

north_america = ['Canada', 'United States of America']

#Northern Europe (nordic)
northern_europe = ['Denmark', 'Estonia', 'Finland', 'Iceland', 'Latvia', 'Lithuania', 'Norway', 'Sweden']
		   
western_europe = ['Belgium', 'France', 'Republic of Ireland', 'Luxembourg', 'Monaco', 'Netherlands', 'United Kingdom']

central_europe = ['Austria', 'Czech Republic', 'Germany', 'Hungary', 'Liechtenstein', 'Poland', 'Slovakia', 'Switzerland']

southern_europe = ['Andorra', 'Italy', 'Malta', 'Portugal', 'San Marino', 'Spain', 'Vatican City']

#Southeastern Europe (mostly Balkan)
southeastern_europe = ['Albania', 'Bosnia and Herzegovina', 'Bulgaria', 'Croatia', 'Cyprus', 'Greece',
          'Kosovo', 'North Macedonia', 'Moldova', 'Montenegro', 'Romania', 'Serbia', 'Slovenia']
		   
eastern_europe = ['Russia', 'Belarus', 'Ukraine']
regions = [subsaharan_africa, menat, south_asia, oceania, latin_america, eastern_europe]

In [135]:
def format_prompt_loc(system_instruction, instruction, input, region):
    # print(input)
    text = input['text']
    label = str(input['label'])
    object = input['object']
    span = input['span']
    loc = random.sample(region,1)
    print(loc)
    message = instruction.format(text = text, object = object, span = span, loc=loc)
    messages = [
        {"role": "system", "content": system_instruction},
        {"role": "user", "content": message},
    ]
    return messages

In [134]:
def request_loc(samples, prompt_type):
    for sample in samples:
        for region in regions:
            print(region)
            messages = format_prompt_loc(system_instruction, prompt_type, sample, region)
            response = openai.ChatCompletion.create(
                model=MODEL,
                response_format={ "type": "json_object" },
                messages= messages,
                temperature=0,
            )
            ans_model = response['choices'][0]['message']['content']
            print(sample)
            print(ans_model)
        print('===================')

Nonce words

In [81]:
nonce_words = ["roagly", "vibble", "drok", "scrop", "plard", "hif", "tepable", "plawic", "bluth", "sprat", "flurf"]

In [87]:
def format_prompt_concepts(system_instruction, instruction, input):
    
    text = input['text']
    label = str(input['label'])
    object = input['object']
    span = input['span']
    nonce = random.sample(nonce_words,1)
    message = instruction.format(text = text, nonce = nonce, object = object, span = span)
    messages = [
        {"role": "system", "content": system_instruction},
        {"role": "user", "content": message},
    ]
    return messages

In [89]:
def request_concepts(samples, prompt_type):
    for sample in samples:
        messages = format_prompt_concepts(system_instruction, prompt_type, sample)
        response = openai.ChatCompletion.create(
            model=MODEL,
            response_format={ "type": "json_object" },
            messages= messages,
            temperature=0,
        )
        ans_model = response['choices'][0]['message']['content']
        # print(messages[1]['content'])
        print(sample)
        print(ans_model)
        print('===================')

# Bias tests

### Frequency bias

In [21]:
frequency_bias_instruction = """Replace one word of higher frequency in English vocabulary with a less frequent word. Add "replaced_word" and "new_word" fields to json output with the replaced and new words, accordingly.

Example:

Text: The cat licked its paw.
Modified Text: The yak licked its paw.


Text: {text}
Object: {object}
Refers to: {span}"""

In [ ]:
request(samples, frequency_bias_instruction)

### Temporal bias

In [23]:
temporal_bias_instruction = """Please replace one word with its old-fashioned synonym. 

Text: {text}
Object: {object}
Refers to: {span}

""" 


In [ ]:
request(samples, temporal_bias_instruction)

### Geographical bias

In [132]:
geographical_bias_instruction = """Change the entity name in the sentence with name commonly used in {loc}. 
Change the other words in the sentence so that it is culturally fitted with context of {loc}.
Do not modify entities mentioned in {span}. Write the modified sentences in English.

Text: {text}
Object: {object}
Refers to: {span}"""

In [ ]:
request_loc(samples, geographical_bias_instruction)

### Position bias

In [30]:
position_bias_instruction = '''Move the text that the pronoun {object} refers to ({span}) to another position in the same sentence.

Text: {text}
Object: {object}
Refers to: {span}

'''

In [ ]:
request(samples, position_bias_instruction)

### Length Bias 

In [34]:
length_bias_instruction = '''Modify the sentence length, but retain sentence meaning. Do not remove or modify the following text: {span}.

1. "shorter_sentence": Remove 1 - 3 words from the sentence.
2. "longer_sentence": Add 2 - 5 words to the sentence.

Text: {text}
Object: {object}
Refers to: {span}

'''

In [ ]:
request(samples, length_bias_instruction)

## Orthography

### Typos

In [36]:
typo_instruction = '''Add typos to the text. Common types of typos are:

1. "addition": Adding a letter: Forty (correct) vs. Fourty
2. "omission": Omitting a letter: Embarrass (correct) vs. Embarass
3. "flipping": flipping letters: Friend (correct) vs. Freind

Reply with the modified text for each type of typo. Do not modify the following words: {object}, {span}

Text: {text}
Object: {object}
Refers to: {span}

'''

In [ ]:
request(samples, typo_instruction)

## Capitalization

In [40]:
capitalization_instruction = '''Modify the text capitalization by:

1. "lower": change a word with upper case to lower case.

2. "upper": change a word starting with lower case to upper case.

3. "all_caps": change a word to ALL CAPS.

4. "sponge": change a word to SpOnGeBoBcASe.

Reply with the modified text for each type of change.

Text: {text}
Object: {object}
Refers to: {span}

'''

In [ ]:
request(samples, capitalization_instruction)

## Punctuation

In [44]:
punctuation_instruction = '''Make change to the punctuation of the text:

1. "addition": add a random comma, semi-colon etc 

2. "replacement": change the full stop to the exclamation or question mark 

3. "glueing": remove white space between two words (glue them together). If possible, glue the word which the pronoun {object} refers to.

Reply with the modified text for each type of change.

Text: {text}
Object: {object}
Refers to: {span}

'''

In [ ]:
request(samples, punctuation_instruction)

## Morphology

## Derivation

In [56]:
derivation_replacement_instruction ='''Find a derived word (a word with a suffix or a prefix) in the text below and replace it with a non-derived word (word without any prefixes or suffixes). Do not modify the following words: {span}.


Example: a sometimes tedious film -> a sometimes dull film (tedious is derived from tedium using a -ios suffix)

Example: a very good film -> Skip

Text: {text}
Object: {object}
Refers to: {span}'''

In [ ]:
request(samples, derivation_replacement_instruction)

In [60]:
antiderivation_replacement_instruction ='''Find any non-derived word (a word without suffixes or prefixes) in the text below and change it into a derived word (word with a prefix or a suffix). Do not add grammatical suffixes (-s, -ed, -er, -est). Do not modify the following words: {span}.

Example: a sometimes dull film ->  a sometimes tedious film (tedious is derived from tedium using a -ios suffix)

Example: an very hard task -> an increasingly hard task (increasingly is derived from increasing using a -ly suffix)

Example: amazing accomplishment -> Skip (both words are already derived)

Text: {text}
Object: {object}
Refers to: {span}'''

In [ ]:
request(samples, antiderivation_replacement_instruction)

## Compound words

In [62]:
compound_word_instruction ='''Find any non-compond (single-root) word in the text below and change it into a compound word (word with several roots). Do not modify the following words: {span}.


Example: 
"a sequence of ridiculous shooting scenes" -> "a sequence of ridiculous shoot-'em-up scenes"
Example: dull acting ->  lacklustre acting

Text: {text}
Object: {object}
Refers to: {span}
'''

In [ ]:
request(samples, compound_word_instruction)

## Irregular verbs

In [64]:
regular_verb_instruction ='''Find a regular verb in the past tense and replace it with an irregular verb.

Example: he received a prize -> he got a prize

Example: amazing stuff  ->  Skip

Text: {text}
Object: {object}
Refers to: {span}'''

In [ ]:
request(samples, regular_verb_instruction)

## Syntax

### Active to passive

In [66]:
passive_voice_instruction = '''Rewrite the sentence in passive voice.

Text: {text}
Object: {object}
Refers to: {span}

'''

In [ ]:
request(samples, passive_voice_instruction)

### Coordinating conjunctions

In [68]:
coordinating_conjunction_instruction = '''Change a noun or verb in this sentence into multiple nouns or verbs combined with coordinating conjunction. Do not modify the following words: {span}

Text: {text}
Object: {object}
Refers to: {span}'''

In [ ]:
request(samples, coordinating_conjunction_instruction)

### Tense

In [70]:
tense_instruction = '''Change the tense of verbs in the sentence. Keep the tenses consistent across the passage.


Text: {text}
Object: {object}
Refers to: {span}'''

In [ ]:
request(samples, tense_instruction)

### Grammatical role

In [72]:
grammatical_role_instruction = '''Modify the position of grammatical role in the sentence.

1. "subject_object": swap the subject with object
Example: 
Miranda asked him a question. --> He asked Miranda a question.

2. "entities": swap the position of other entities
Example:
Samantha, the older Rico's friend, will be appointed as Chair of Student Body replacing Marie. --> Marie, the older Samantha's friend, will be appointed as Chair of Student Body replacing Rico.

Text: {text}
Object: {object}
Refers to: {span}'''

In [ ]:
request(samples, grammatical_role_instruction)

### Clause structure

In [74]:
sentence_structure_instruction = '''
Change the position of main and subordinate clause. If the input is simple sentence, skip.

Text: {text}
Object: {object}
Refers to: {span}'''

In [ ]:
request(samples, sentence_structure_instruction)

## Semantics and lexicon

## Concept replacement

In [84]:
concept_replacement_instruction ='''Replace a concept with one that is similar but fail the quality. Do not modify the following words: {span}

Replacement types include:

1. "synonym": synonyms
Example: embodies the character with an effortlessly regal charisma . -> embodies the character with an effortlessly regal charm. (charm and charisma are synonyms)

2. "hierarchy": hyper/hyponyms
Example: The title not only describes its main characters, but the lazy people behind the camera as well. -> The title not only describes its main characters, but the lazy people behind the equipment as well. (equipment is a hypernym of camera)

3. "nonce": the following nonce word: {nonce}
Example: Has a lot of the virtues of eastwood at his best. - Has a lot of the virtues of bluth at his best. (bluth is a nonce word)

4. "idiom": metaphors/idioms
Example: This is a train wreck of an action film -> This is a disastrous action film (train wreck is a metaphor for disastrous)

Reply the modified text for each type of replacement.

Text: {text}
Object: {object}
Refers to: {span}

'''

In [ ]:
request_concepts(samples, concept_replacement_instruction)

## Adjectives/adverbs change

In [95]:
adjective_adverb_remove_instruction = '''Remove an adjective or adverb from the sentence. Do not remove the following words: {object}, {span}

Text: {text}
Object: {object}
Refers to: {span}
'''

In [ ]:
request(samples, adjective_adverb_remove_instruction)

In [93]:
adjective_adverb_add_instruction = '''Add an adjective or an adverb to the sentence.

Text: {text}
Object: {object}
Refers to: {span}
'''

In [ ]:
request(samples, adjective_adverb_add_instruction)

## Pragmatics and discourse

### Negation

In [97]:
negation_instruction = '''Negate the text by making minimal modifications to introduce different types of negation. The types of negation include:

1. "verbal": verbal negation: when the negation is grammatically associated with the verb, the head of the clause.
Examples:
He trusts them. => He does not trust them.

2. "absolute": Absolute negator: no (including compounds nobody, nothing, etc., and the independent form none), neither, nor, never.
Example:
He trusts them. => He trusts noone.

3. "approximate": Approximate negators: few, little; barely, hardly, scarcely; rarely, seldom.
Example:
He trusts people. => He rarely trusts people.

4. "affixal": Affixal negators: un-, in-, non-, -less, etc. Do not change the root of the word you add the affix to.
Example:
He is healthy => He is unhealthy.

5. "lexical": Lexical negation: when the negation is added by substituting the main predicate of the sentence with its antonym or word carrying negative meaning.
Examples:
The house is big. => The house is small.

6. "unimportant": Unimportant negation: when the negation does not affect the main clause of the text and does not affect the coreference chain.
Examples:
A man is driving his car. => A shirtless man is driving his car.

7. "double": Double negation: when there are two instances of negation that cancel each other and the meaning is affirmative.

Reply with the modified text for each type of negation.

Text: {text}
Object: {object}
Refers to: {span}
'''

In [ ]:
request(samples, negation_instruction)

In [100]:
antinegation_instruction = '''Check if the text contains negation, and if it does, remove it. Do not make any other modifications. If there is no negation, output Skip. 


Examples:  

Text: The house is not pretty. 

Modified text: The house is pretty.  

 

Text: The story didn't leave anyone unaffected. 

Modified text: The story left everyone unaffected. 


Text: The house is pretty. 

Modified text: Skip.  


Text: {text}
Object: {object}
Refers to: {span}''' 

In [ ]:
request(samples, antinegation_instruction)

## Discourse markers

In [102]:
discourse_instruction = '''Make modification to the discourse makers in the text. The types of modification include:

1. "addition": Add discourse markers to the sentence.
Example:
He was hungry, he went out to eat => He was hungry, so he went out to eat.

2. "replacement": Change the discourse marker into a different one. If there is no discourse marker, output Skip.
Example:
He was hungry although he had dinner => He was hungry so he had dinner. (although => so)

3. "deletion": Delete the discourse marker. If there is no discourse marker, output Skip.
Example:
He was hungry so he had dinner => He was hungry, he had dinner.

For each type of modification, reply with the modified sentences.

Text: {text}
Object: {object}
Refers to: {span}

'''

In [ ]:
request(samples, discourse_instruction)

## Sentiment

In [104]:
sentiment_instruction = '''Add a word or a phrase with a positive or negative sentiment to the sentence. 

Example: We beat the competition -> We are happy to beat the competition. 

Text: {text}
Object: {object}
Refers to: {span}

'''

In [ ]:
request(samples, sentiment_instruction)

## Style

### Casual vs formal style

In [106]:
casual_instruction = '''Rewrite the sentence in informal way. Do not modify the following words: {span}

Text: {text}
Object: {object}
Refers to: {span}
'''

In [ ]:
request(samples, casual_instruction)

### Simple vs Complex English

### Plain English

In [110]:
plain_english_instruction = '''Please rewrite the sentence in order to make it easier to understand by non-native speakers of English. 
You can do so by replacing complex words with simpler synonyms (i.e. paraphrasing), deleting unimportant information (i.e. compres-sion), and/or splitting a long complex sentence into several simpler ones. The final simplified sentence needs to be grammatical, fluent, and retain the main ideas of its original counterpart without altering its meaning.
Do not modify the following words: {span}

Text: {text}
Object: {object}
Refers to: {span}

'''

In [ ]:
request(samples, plain_english_instruction)

### Complex English

In [112]:
complex_english_instruction = '''Transform the original sentence into more complex English. 
Change some words with their more sophisticated technical synonym.
Change the sentence structure into more complex one.
Keep the modified sentence has nearly similar length with original one and do not alter its meaning.
Do not modify the following words: {span}

Text: {text}
Object: {object}
Refers to: {span}

'''

In [ ]:
request(samples, complex_english_instruction)

### Dialectal features

#### African American English

In [114]:
aave_english_instruction = '''Rewrite the text in African American Vernacular English. Do not modify the following words: {span}.

Text: {text}
Object: {object}
Refers to: {span}

'''

In [ ]:
request(samples, aave_english_instruction)

In [116]:
singlish_instruction = '''Rewrite the text in Singapore Colloquial English (Singlish, Basilect). Do not modify the following words: {span}.

Text: {text}
Object: {object}
Refers to: {span}

'''

In [ ]:
request(samples, singlish_instruction)

#### Indian English 

In [118]:
higlish_instruction = '''Rewrite the text using Indianisms and Indian English grammar. Do not modify the following words: {span}.

Text: {text}
Object: {object}
Refers to: {span}

'''

In [ ]:
request(samples, higlish_instruction)